In [20]:
import torch
import numpy as np
from konlpy.tag import Okt
import re
import pickle
from torch import nn

In [21]:
# 문장 분류하는 모델
class SentenceClassifier(nn.Module):
    def __init__(self, n_vocab, hidden_dim, embedding_dim, n_layers, n_classes ,dropout=0.5, bidirectional=True, model_type="lstm"):
        super().__init__()  # 부모클래스 상속

        self.embedding = nn.Embedding(num_embeddings=n_vocab, embedding_dim=embedding_dim, padding_idx=0)

        # rnn모델 일 경우
        if model_type == 'rnn':
            self.model = nn.RNN(
                input_size=embedding_dim, hidden_size=hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout, batch_first=True
            )
        # lstm모델 일 경우
        elif model_type == 'lstm':
            self.model = nn.LSTM(
                input_size=embedding_dim, hidden_size=hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout, batch_first=True
            )

        # bidirectional은 양방향성을 의미하는 파라미터
        if bidirectional:
            self.classifier = nn.Linear(hidden_dim * 2, n_classes)   # 양방향일때 타임스탭에서 양방향의 정보(순방향,역방향)의 출력들을 결합하여 분류기에 전달
        else:
            self.classifier = nn.Linear(hidden_dim, n_classes)
        self.dropout = nn.Dropout(dropout)

    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        output, _ = self.model(embeddings) 
        last_output = output[:, -1, :]
        last_output = self.dropout(last_output)
        logits = self.classifier(last_output)
        return logits

In [2]:
# 1. 새로운 리뷰 데이터 불러오기
new_reviews = ["정말 재미있었어요!", "영 별로였어요..."]

In [3]:
# 2. 한글만 남기고 정규식 적용
def re_text(text):
    text = re.sub(r'[^\n가-힇\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [4]:
new_reviews = [re_text(review) for review in new_reviews]

In [5]:
# 3. Okt 토크나이저 로드 및 토큰화
tokenizer = Okt()
STOP_PATH = 'kor_stopwords.txt'

def make_stopwords(STOP_PATH):
    with open(STOP_PATH, 'r', encoding='utf-8') as f:
        stopwords = f.read().splitlines()
    return set(stopwords)

In [7]:
stopwords = make_stopwords(STOP_PATH)
new_tokens = [[token for token in tokenizer.morphs(review) if token not in stopwords] for review in new_reviews]

In [13]:
# 4. 단어 사전 로드
with open('vocab.pkl', 'rb') as f:
    vocab_list = pickle.load(f)

# 리스트를 사전으로 변환
token_to_id = {token: idx for idx, token in enumerate(vocab_list)}

unk_id = token_to_id.get("<unk>", 1)
pad_id = token_to_id.get("<pad>", 0)

In [14]:
# 5. 정수 인코딩
new_ids = [[token_to_id.get(token, unk_id) for token in tokens] for tokens in new_tokens]

In [15]:
# 6. 패딩 적용
max_length = 120
def pad_sequences(sequences, max_length, pad_value):
    result = []
    for sequence in sequences:
        sequence = sequence[:max_length]
        pad_length = max_length - len(sequence)
        padded_sequence = sequence + [pad_value] * pad_length
        result.append(padded_sequence)
    return np.asarray(result)

In [16]:
new_ids_padded = pad_sequences(new_ids, max_length, pad_id)

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.load('best_model.pth', map_location=device)
model = model.to(device)
model.eval()

SentenceClassifier(
  (embedding): Embedding(10002, 128, padding_idx=0)
  (model): LSTM(128, 128, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (classifier): Linear(in_features=256, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [23]:
# 8. 모델 예측
new_ids_tensor = torch.tensor(new_ids_padded).to(device)  # 데이터를 모델이 있는 디바이스로 이동

with torch.no_grad():
    outputs = model(new_ids_tensor)
    predictions = torch.sigmoid(outputs)

# 9. 예측 결과 출력 (0.5 이상이면 긍정, 미만이면 부정)
for i, review in enumerate(new_reviews):
    prediction = 1 if predictions[i] >= 0.5 else 0
    print(f"리뷰: {review}")
    print(f"예측된 감정: {'긍정' if prediction == 1 else '부정'}")

리뷰: 정말 재미있었어요
예측된 감정: 부정
리뷰: 영 별로였어요
예측된 감정: 부정
